In [ ]:
# HAND DETECTION CODE 
import cv2
import mediapipe as mp

class HandDetector:
    def __init__(self, max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5): #The __init__ method initializes an instance of the 
        #HandDetector class with default parameters for detecting hands and setting up drawing specifications.
        
        
        self.max_num_hands = max_num_hands
        self.min_detection_confidence = min_detection_confidence
        self.min_tracking_confidence = min_tracking_confidence   #Stores the input parameters as instance variables for later use within the class methods.
        
        self.mp_hands = mp.solutions.hands #Initializes the MediaPipe hands module, which provides hand detection and tracking functionality.

        self.hands = self.mp_hands.Hands(static_image_mode = False,
                                         max_num_hands=self.max_num_hands, 
                                         min_detection_confidence=self.min_detection_confidence, 
                                         min_tracking_confidence=self.min_tracking_confidence)
        
        self.mp_drawing = mp.solutions.drawing_utils #Initializes the MediaPipe drawing utilities module, which provides functions for drawing landmarks and connections on the image.
        
        self.landmark_spec = self.mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2)  # Red landmarks
        self.connection_spec = self.mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)  # Blue connections

    def findHands(self, image, draw=True):
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert the frame to RGB
        self.results = self.hands.process(image_rgb)  # Process the frame and find hands

        if self.results.multi_hand_landmarks: #contains the landmark data for all detected hands. If this attribute is not None, it means that hands have been detected.
            for hand_landmarks in self.results.multi_hand_landmarks:
                
                if draw:      #If the draw flag is True, it draws the landmarks and connections using the MediaPipe drawing utilities.
                    self.mp_drawing.draw_landmarks(    #uses the MediaPipe drawing utilities
                        image, hand_landmarks, self.mp_hands.HAND_CONNECTIONS, self.landmark_spec, self.connection_spec)

        return image

    def findPosition(self, image, draw=True):
        all_hand_positions = []
        if self.results.multi_hand_landmarks:
            for handNo, hand_landmarks in enumerate(self.results.multi_hand_landmarks):
                hand_positions = []
                for id, lm in enumerate(hand_landmarks.landmark): #provides both the index (id) and the landmark object (lm) for each landmark.
                    h, w, c = image.shape   #height (h), width (w), and number of color channels (c) 
                    cx, cy = int(lm.x * w), int(lm.y * h)  #Converts the normalized coordinates (lm.x, lm.y) to pixel coordinates
                    hand_positions.append([id, cx, cy, lm.z]) #id: Landmark index , cx: X-coordinate in pixels , cy: Y-coordinate in pixels , lm.z: Z-coordinate (depth) in normalized space

                    if draw:
                        cv2.circle(image, (cx, cy), 5, (255, 0, 0), cv2.FILLED)
                all_hand_positions.append(hand_positions)  #Appends the hand_positions list (containing all landmark positions for the current hand) to all_hand_positions
        return all_hand_positions

def main():
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)

    if not cap.isOpened():
        print("Error: Could not open video capture")
        exit()

    detector = HandDetector()

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame from the camera.")
            break

        frame_flipped = cv2.flip(frame, 1)  # Flip the frame horizontally

        frame_flipped = detector.findHands(frame_flipped)
        hand_positions = detector.findPosition(frame_flipped)

        # Display the frame 
        cv2.imshow("Image", frame_flipped)

        key = cv2.waitKey(1)
        if key == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()